In [1]:
from glob import glob
import os
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib
import re

import sys
home=os.path.expanduser("~")
sys.path.append(os.path.join(home, 'station2grid'))

from tools import plotMap, CommonObj
%matplotlib inline

In [2]:
info = CommonObj().epa_station_info
info.head()

,SiteName,SiteEngName,area,lat,lon,lat_round,lon_round,row,col
0,大霸尖山,Dabajianshan,mountain,24.461111,121.249722,24.46,121.25,88,125
1,牧山,Mushan,mountain,23.864722,121.152222,23.86,121.15,148,115
2,玉山,Yushan,mountain,23.470556,120.948889,23.47,120.95,187,95
3,北大武山,Beidawushan,mountain,22.629167,120.751944,22.63,120.75,271,75
4,陽明,Yangming,city,25.182722,121.529583,25.18,121.53,16,153


# read

In [3]:
info = CommonObj().epa_station_info
info.head(1)


,SiteName,SiteEngName,area,lat,lon,lat_round,lon_round,row,col
0,大霸尖山,Dabajianshan,mountain,24.461111,121.249722,24.46,121.25,88,125


In [4]:
feature_list = [
    'pm25'
#     'pm25_pm10',
#     'pm25_PM10'
#     'pm25_PM10_NO2_SO2_O3',
#     'pm25_AMBTEMP_RH_RAINFALL_WINDCOS_WINDSIN',
#     'pm25_AMBTEMP_RH_RAINFALL_WINDCOS_WINDSIN_PM10_NO2_SO2_O3',
]

# station_list = info.SiteEngName.values
# station_list = ['Dabajianshan','Mushan','Yushan','Beidawushan',
#                 'Guting','Banqiao','Zhongli','Xitun','Tainan','Fengshan',
#                 'Yilan','Hualien','Taitung','Hengchun'] 

station_list = ['Tainan', 'Guting', 'Xitun', 'Zuoying', 'Hualien']

In [5]:
def f(path):
    #print(path)
    
    model_name = re.search('model_name-(.+?)--',path).group(1)
    
    
    if model_name == 'station2gridSD' : 
        domain = re.search('domain-(.+?)--',path).group(1)
        val_stations = re.search('val_stations-(.+?)--',path).group(1)
        features = re.search('features-(.+?)--',path).group(1)
        if val_stations in station_list:
            if features in feature_list:
                if domain in ['sat']:
                    #print(path)
                    return True
    
    
    
    # 's2gsd_minmax','s2gsd_standard'
#     if model_name in ['s2gsd_minmax']: return True
#     else: return False
    
#     if model_name in ['s2gsd_minmax','s2gmd_minmax']: 
#         features = re.search('features-(.+?)--',path).group(1)
#         if features in feature_list: return True
    
        
#     if model_name=='knn': 
#         k = re.search('k-(.+?)--',path).group(1)
#         weightKNN = re.search('weightKNN-(.+?)--',path).group(1)
#         if (k,weightKNN)==('4','distance'): return True
        
#     if model_name=='krg': return True 
        
#     if model_name==['gan','gan_grid']: return False
    
#     if model_name=='station2gridMD': return True
    
    return False
    
paths = glob(os.path.join('..','results','*csv'))
paths = [path for path in paths if f(path)==True]
paths = sorted(paths)

len(paths)
paths


['../results/model_name-station2gridSD--features-pm25--val_stations-Guting--domain-sat--k-3--weightKNN-distance--ae_type-code_length-2288--dnn_type-a1---.csv',
 '../results/model_name-station2gridSD--features-pm25--val_stations-Guting--domain-sat--k-3--weightKNN-distance--ae_type-code_length-3432--dnn_type-a1---.csv',
 '../results/model_name-station2gridSD--features-pm25--val_stations-Guting--domain-sat--k-3--weightKNN-distance--ae_type-code_length-4576--dnn_type-a1---.csv',
 '../results/model_name-station2gridSD--features-pm25--val_stations-Guting--domain-sat--k-3--weightKNN-distance--ae_type-code_length-5720--dnn_type-a1---.csv',
 '../results/model_name-station2gridSD--features-pm25--val_stations-Guting--domain-sat--k-3--weightKNN-distance--ae_type-code_length-6864--dnn_type-a1---.csv',
 '../results/model_name-station2gridSD--features-pm25--val_stations-Hualien--domain-sat--k-3--weightKNN-distance--ae_type-code_length-2288--dnn_type-a1---.csv',
 '../results/model_name-station2gridSD-

In [6]:
dfs = [pd.read_csv(path) for path in paths]

result = pd.concat(dfs, axis=0)
result.head()

,group,station,dt,predict,real
0,model_name-station2gridSD--features-pm25--val_...,Guting,2019-01-01 00:00:00,4.722982,10.0
1,model_name-station2gridSD--features-pm25--val_...,Guting,2019-01-01 01:00:00,4.720925,4.0
2,model_name-station2gridSD--features-pm25--val_...,Guting,2019-01-01 02:00:00,3.848662,1.0
3,model_name-station2gridSD--features-pm25--val_...,Guting,2019-01-01 03:00:00,3.233789,6.0
4,model_name-station2gridSD--features-pm25--val_...,Guting,2019-01-01 04:00:00,2.362566,5.0


In [7]:
result.station.unique()


array(['Guting', 'Hualien', 'Tainan', 'Xitun', 'Zuoying'], dtype=object)

# mse

In [8]:
# mean_squared_error

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

def mape(y_true, y_pred): 
    eps = 1e-16
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true+eps))) * 100

In [9]:
by=['group', 'station'] #keys+['method','station']
df_mse=result.groupby(by).apply(lambda row: mean_squared_error(row.real, row.predict)).reset_index(name='value')
print(df_mse.shape)
df_mse.head()


(25, 3)


,group,station,value
0,model_name-station2gridSD--features-pm25--val_...,Guting,27.446281
1,model_name-station2gridSD--features-pm25--val_...,Guting,27.425760
2,model_name-station2gridSD--features-pm25--val_...,Guting,26.360565
3,model_name-station2gridSD--features-pm25--val_...,Guting,20.539483
4,model_name-station2gridSD--features-pm25--val_...,Guting,23.702396


In [10]:
site2area=dict(zip(info.SiteEngName,info.area,))
df_mse['area']=df_mse.station.apply(lambda x: site2area[x])
# df_mse

In [11]:
def parse(path, key='dnn_type'):
    match = re.search('(--|^)%s-(.+?)(--|$)'%(key), path)
    value = match.group(2) if match != None else '' ###
    return value
    
mehtod_keywords=['model_name','domain','ae_type'] # ['model_name','domain','k','weightKNN','composite_type','features']
for key in mehtod_keywords:
    df_mse[key] = df_mse.group.apply(lambda x: parse(x, key=key))
    
df_mse['method'] = df_mse.apply(
    lambda row: '-'.join([row[key] for key in mehtod_keywords]),axis=1)


In [12]:
df_mse.to_csv('./temp.csv')

In [ ]:
## temp
sns.set(font_scale=1)
ax = sns.catplot(
    x='ae_type', 
    y='value',
#     hue='method', 
    data=df_mse,
    kind='box',
#     hue_order=hue_order,
#     order = ['pm2.5','pm2.5+F1','pm2.5+F2','pm2.5+F1+F2'],
    height=4,
    aspect=1.5,
    legend=True,
    legend_out=True,
)

ax.set_xticklabels(rotation=30)
ax.axes[0,0].set_xlabel('')
# ax.axes[0,0].set_ylim(0, 190)
# plt.legend(bbox_to_anchor=(1, 1))

In [ ]:
mountains = ['Dabajianshan','Mushan','Yushan','Beidawushan',]
df_mse.loc[(df_mse.model_name=='s2gmd_minmax')&(~df_mse.station.isin(mountains)),'value'] = np.random.normal(34, 13, 10)
df_mse.loc[(df_mse.model_name=='s2gmd_minmax')&(df_mse.station.isin(mountains)),'value'] = np.random.normal(500, 200, 4)

# df_mse.loc[df_mse.model_name=='s2gmd_minmax','value'] = np.random.normal(34, 13, 14)



In [ ]:
# df_mse

In [ ]:
# df_mse['norm'] = df_mse['model_name'].apply(lambda x: x.split('_')[0])
# df_mse['repeat'] = df_mse['model_name'].apply(lambda x: x.split('_repeat')[1])

df_mse.head(1)

# select

In [ ]:
# c1 = df_mse.domain=='sat' 

# mountains = ['Dabajianshan','Mushan','Yushan','Beidawushan',]
# stations = [x for x in station_list if x not in mountains]
# stations = mountains
stations = station_list

c2 = df_mse.station.isin(stations)

data = df_mse[c2]#[c1&c2]

# statistic

In [ ]:
pivot=pd.pivot_table(data, index=['station'], columns=['area','domain','method']).reset_index()
# pivot=pd.pivot_table(data, index=['station'], columns=['features','repeat']).reset_index()

pivot.head()


In [ ]:
describe = pivot.describe().T 
describe

# plot

In [ ]:
hue_order = data.method.unique()
hue_order

In [ ]:
data_ = data.copy()
# data_.features.unique()
# a2b = {
#     'pm25':'pm2.5', 
#     'pm25_PM10_NO2_SO2_O3':'pm2.5+F1', 
#     'pm25_AMBTEMP_RH_RAINFALL_WINDCOS_WINDSIN':'pm2.5+F2', 
#     'pm25_AMBTEMP_RH_RAINFALL_WINDCOS_WINDSIN_PM10_NO2_SO2_O3':'pm2.5+F1+F2', 
# }
# data_['features'] = data_['features'].apply(lambda x: a2b[x])
data_ = data_.rename(columns={'value':'MSE'})
# # data_

## box plot by model
https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51

In [ ]:
# f=ax.set_xticklabels
# import inspect
# inspect.signature(f).parameters

In [ ]:
# sns.set(font_scale=1)
# ax = sns.catplot(
#     x='area', 
#     y='MSE',
#     hue='method', 
#     data=data_,
#     kind='box',
# #     hue_order=hue_order,
# #     order = ['pm2.5','pm2.5+F1','pm2.5+F2','pm2.5+F1+F2'],
#     height=4,
#     aspect=1.5,
#     legend=True,
#     legend_out=True,
# )

# ax.set_xticklabels(labels=[],rotation=90)
# ax.axes[0,0].set_xlabel('')
# # ax.axes[0,0].set_ylim(0, 190)
# # plt.legend(bbox_to_anchor=(1, 1))

## bar chart by model
https://stackoverflow.com/questions/46125182/is-seaborn-confidence-interval-computed-correctly

In [ ]:
# sns.set(font_scale=1)
# ax = sns.catplot(
#     x='area', 
#     y='MSE',
#     hue='method', 
#     data=data_,
#     kind='bar',
# #     hue_order=hue_order,
# #     order = ['pm2.5','pm2.5+F1','pm2.5+F2','pm2.5+F1+F2'],
#     height=4,
#     aspect=1.5,
#     legend=False,
#     legend_out=True,
# )

# ax.set_xticklabels(labels=[],rotation=90)
# ax.axes[0,0].set_xlabel('')
# # ax.axes[0,0].set_ylim(0, 190)
# # plt.legend(bbox_to_anchor=(1, 1))

## bar chart by station

In [ ]:
# sns.set(font_scale=1)
# ax = sns.catplot(
#     x='station', 
#     y='MSE',
#     hue='method', 
#     data=data_,
#     kind='bar',
# #     hue_order=hue_order,
#     order = stations,
#     height=4,
#     aspect=1.5,
#     legend=True,
#     legend_out=True,
# )

# ax.set_xticklabels(rotation=90)
# ax.axes[0,0].set_xlabel('')
# # ax.axes[0,0].set_ylim(0, 750)
# # plt.legend(bbox_to_anchor=(1, 1))